In [19]:
#Understanding necessary Astropy tools
from astropy import units as u 
from astropy import constants as cst 
from astropy.coordinates import SkyCoord
from astropy.table import QTable, Table

import numpy as np 

#Example of use: Units and Constants 
g_force  = (cst.G * cst.M_sun * cst.M_earth) / (u.au)**2
print(f'Gravitational force beetwen Sun and Earth: {g_force.to(u.N)}')

#Example of use: Coordinates 
position = SkyCoord(45.5*u.hourangle, 23.9*u.deg)
print(f'Normalized coordinates in degrees: {position}')
print(f'Normalized coordinates in hourangles: {position.ra.hms}')

#Example of use: QTable 
planets = ['Venus', 'Earth', 'Mars']
order = np.array([2,3,4])
my_rate = [8.8,10.0,4.5]
masses = [cst.M_earth*0.815, cst.M_earth, cst.M_earth*0.1075] #unit=kg
rock_planets_table = QTable([planets, order, masses, my_rate],
                             names=('Planets', 'Order', 'Masses', 'My_Rate'),
                               meta={'name':'Test Table'})

print(rock_planets_table)

Gravitational force beetwen Sun and Earth: 3.541545424043141e+22 N
Normalized coordinates in degrees: <SkyCoord (ICRS): (ra, dec) in deg
    (322.5, 23.9)>
Normalized coordinates in hourangles: hms_tuple(h=np.float64(21.0), m=np.float64(29.0), s=np.float64(59.99999999998721))
Planets Order         Masses        My_Rate
                        kg                 
------- ----- --------------------- -------
  Venus     2 4.867316812249974e+24     8.8
  Earth     3 5.972167867791379e+24    10.0
   Mars     4 6.420080457875733e+23     4.5


In [130]:
from astroquery.simbad import Simbad
from astroquery.sdss import SDSS

#Example of Use: Astroquery
target_object = 'NGC 5548'
Simbad.add_votable_fields('rvz_redshift', 'otype', 'dim')
galaxy_table = Simbad.query_object(target_object)
print(galaxy_table)

ra_coord = galaxy_table['ra']
dec_coord = galaxy_table['dec']
galaxy_coord = SkyCoord(ra_coord, dec_coord)
#print(galaxy_coord)

#Testing Galaxy Diameter Usability 
if galaxy_table['galdim_majaxis']/2 * u.arcmin < 3 * u.arcmin:
    galaxy_rad = galaxy_table['galdim_majaxis']/2 * u.arcmin
else:
    galaxy_rad = 2.9 * u.arcmin

#Searching for Spectra
galaxy_data = SDSS.query_region(galaxy_coord, radius = galaxy_rad, spectro = True)

while galaxy_data is None and galaxy_rad < 2.9 * u.arcmin:
    galaxy_rad = galaxy_rad + 0.1 * u.arcmin 
    galaxy_data = SDSS.query_region(galaxy_coord, radius = galaxy_rad, spectro = True)

if galaxy_data is None : 
    print('No spectra were found whitin the 3 arcmin limit.')

else:
    print(f'({len(galaxy_data)} spectra were found!)')
    print(galaxy_data.colnames)

    plate = galaxy_data['plate'][0]  
    mjd = galaxy_data['mjd'][0]
    fiberID = galaxy_data['fiberID'][0]


 main_id          ra              dec       ... galdim_majaxis matched_id
                 deg              deg       ...     arcmin               
--------- ------------------ -------------- ... -------------- ----------
NGC  5548 214.49808345492997 25.13683420063 ...        1.48647  NGC  5548
(1 spectra were found!)
['ra', 'dec', 'objid', 'run', 'rerun', 'camcol', 'field', 'z', 'plate', 'mjd', 'fiberID', 'specobjid', 'run2d']
